In [2]:
import loader
data_user = loader.load('./data/userMini.json')


In [3]:
data_business = loader.load('./data/businessMini.json')

In [5]:
def index_category_builder(business):
    index_category = {}
    index = 0
    for v in business.values():
        for category in v['categories']:
            if category not in index_category.keys():
                index_category[category] = index
                index += 1
    return index_category
index_category = index_category_builder(data_business)


In [13]:
## get index_user and array_user
def index_user_builder(user, business):
    index_user = {}
    array_user = []
    index = 0
    for k, v in user.items():
        index_user[k] = index
        index += 1
        tmp = [0 for i in range(len(index_category))]
        for businessId in v:
            for category in business[businessId]['categories']:
                tmp[index_category[category]] += 1
        array_user.append(tmp)
    return index_user, array_user
index_user, array_user = index_user_builder(data_user, data_business)


In [ ]:
##get array_user
def array_user_builder(user, index_category):
    array_user = []
    for v in user.values():
        tmp = [0 for i in range(len(index_category))]
        for businessId in v:
            tmp[index_category[businessId]] += 1
        array_user.append(tmp)
    return array_user
array_user = array_user_builder(data_user, index_category)

In [17]:
del data_business, data_user

In [18]:
## 余弦相似度的距离
def get_dis(array):
    from sklearn.metrics.pairwise import pairwise_distances
    dis = pairwise_distances(array,metric="cosine")
    return dis
dis = get_dis(array_user)

